# ResNet50V2
### Modelo ResNet50V2 utilizando apenas a imagem

In [1]:
import numpy as np
import pandas as pd
from PIL import Image
import keras
import tensorflow as tf
from keras import layers
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


## Rede

In [2]:
resnet_cnn = tf.keras.applications.ResNet50V2(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation='softmax'
)

In [3]:
print("trainable_weights:", len(resnet_cnn.trainable_weights))
print("non_trainable_weights:", len(resnet_cnn.non_trainable_weights))

resnet_cnn.trainable = False; print('------------')

print("trainable_weights:", len(resnet_cnn.trainable_weights))
print("non_trainable_weights:", len(resnet_cnn.non_trainable_weights))

trainable_weights: 172
non_trainable_weights: 98
------------
trainable_weights: 0
non_trainable_weights: 270


In [4]:
inputs = keras.Input(shape=(224, 224, 3), batch_size=32)
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = resnet_cnn(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

## Dados

In [5]:
batch_size = 32
img_height = 224
img_width = 224
data_dir = 'dados/imgs/'

In [6]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 208724 files belonging to 2 classes.
Using 166980 files for training.


In [7]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 208724 files belonging to 2 classes.
Using 41744 files for validation.


Pensar em utilizar uma camada de normalização
```
normalization_layer = tf.keras.layers.Rescaling(1./255)
```


```
keras.metrics.F1Score(average=None, threshold=None, name="f1_score", dtype=None)
```

## Treino

In [8]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[keras.metrics.BinaryAccuracy()])

In [ ]:
model.fit(train_ds, epochs=1, validation_data=val_ds)

In [11]:
model.save_weights('resnet_models/model_v1.weights.h5')

## Resultados

In [14]:
test_dataset = tf.keras.utils.image_dataset_from_directory(
          'test/',
           shuffle=False,
           batch_size=batch_size,
           image_size=(224,224))

Found 89925 files belonging to 1 classes.


In [16]:
results = model.predict(test_dataset)

 918/2811 ━━━━━━━━━━━━━━━━━━━━ 27:48 881ms/step